## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)


root_path = './'

models_path = os.path.join(root_path, 'saved_models_mnist_v9')
if not os.path.exists(models_path):
    os.mkdir(models_path)


mnist = keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train / 127.5) - 1, (x_test / 127.5) - 1


BATCH_SIZE = 256

Using TensorFlow backend.


1.13.1


In [2]:
print(x_train.shape, y_train.shape)
x_train = np.expand_dims(x_train, -1)
y_train = to_categorical(y_train)
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)
(60000, 28, 28, 1) (60000, 10)


In [3]:
def custom_loader():

  trainset_size = x_train.shape[0]

  p = 0
  while True:

    idx_from = (p * BATCH_SIZE) % trainset_size

    idx_to = idx_from + BATCH_SIZE

    batch_x = x_train[idx_from: idx_to]
    batch_y = y_train[idx_from: idx_to]
    
    p += 1
  
    yield batch_x, batch_y

In [4]:
custom_gen = custom_loader()
for i in range(3):
  batch_x, batch_y = next(custom_gen)
  print(batch_x.shape, batch_y.shape)

(256, 28, 28, 1) (256, 10)
(256, 28, 28, 1) (256, 10)
(256, 28, 28, 1) (256, 10)


## Model Design

In [ ]:
enc_len = 0
age_len = 10
img_shape = (28, 28, 1)
width, height, depth = (28, 28, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 50  # 32 x 32 x 3
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    print("D M1:")
    conv.summary()
    
    model = keras.Sequential([
        # output
        keras.layers.Dense(1, input_shape=(age_len+12,)),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    clf = keras.Sequential([
        # output
        keras.layers.Dense(age_len, input_shape=(age_len+12,)),
        keras.layers.Activation(tf.nn.softmax),
    ])
    
    print("D M2:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
    inputs = keras.layers.concatenate([c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    # 0, 1, 2, 3, .. 9
    lbl = clf(inputs)
    
    return keras.models.Model([c2, z], [outputs, lbl])


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(1, 1),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    print("G M2:")
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    print("G M1:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c2, x])
    
    # flat dense output
    out_1 = model(inputs)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model([c2, x], outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
D M1:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 13, 13, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 6, 32)          128       
________________________________________________________________

In [ ]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 60)           0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 11, 11, 1)    7381        concatenate_2[0][0]              
__________

In [ ]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2, 2, 3)      5859        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 12)           0           sequential_1[1][0]               
__________________________________________________________________________________________________
concatenat

## Generative Adversarial Network

In [ ]:
GLR = 0.01  # generator
DLR = 0.01  # discriminator


# Wasserstein
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=[keras.losses.binary_crossentropy, keras.losses.categorical_crossentropy],
    metrics=['accuracy']
)


# condition
# c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c2, x])
out, lbl = discriminator([c2, z])

# GAN
gan = keras.models.Model(inputs=[c2, x], outputs=[out, lbl])

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=[keras.losses.binary_crossentropy, keras.losses.categorical_cross_entropy],
    metrics=['accuracy'])

In [ ]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 28, 28, 1)    16662       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            5882        input_5[0][0]                    
          

## Visualization Method

In [ ]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_mnist-v20')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [ ]:
def visualizeGAN(e, z_real, z_fake, conditions=None):

    fig, axes = plt.subplots(2, 5, figsize=(20, 10))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0 and z_real is not None:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 5 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
                cell.set_title(
                    str(
                        np.argmax(
                            conditions[r_fake * 5 + col]
                        )
                    ) + ": " + str(
                        conditions[r_fake * 5 + col]
                    )
                )

            cell.axis("off")

        if row % 2 == 0 and z_real is not None:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [ ]:
def load_noise():
    
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_true = y_true.eval()

        while True:
            batch_x, batch_y = next(custom_gen)

            sz = batch_x.shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c2 = tf.cast(batch_y, tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield c2, x, y_true


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_true = y_true.eval()

        while True:
            batch_x, batch_y = next(custom_gen)

            sz = batch_x.shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c2 = tf.cast(batch_y, tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict([c2, x])

            # real data
            c2 = tf.cast(batch_y, tf.float32).eval()
            z_real = batch_x
                        
            yield c2, x, z_fake, y_fake, z_real, y_true

## Train Model

In [ ]:
EPOCHS = 2000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# fixed test sample
noise = np.random.normal(size=(age_len, noise_len,) )
conditions = np.identity(age_len)


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
    
        # train
        l1, acc_1, l2, acc_2, loss_1 = discriminator.train_on_batch([c2, z_real], [y_real, c2])
        l3, acc_3, l4, acc_4, loss_2 = discriminator.train_on_batch([c2, z_fake], [y_fake, c2])

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_3)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

      c2, x, y_true = next(gen_itr)

      # train
      l1, acc_1, l2, acc_2, loss_1 = gan.train_on_batch([c2, x], [y_true, c2])

      loss.append(loss_1)
      acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, D:[ACC: %{:.2f}, LOSS: {:.2f}], G:[ACC: %{:.2f}, LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 25 == 0:
        ## visualize results
        synthesized = generator.predict([conditions, noise])
        visualizeGAN(e, None, synthesized, conditions)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, D:[ACC: %85.99, LOSS: 30.14], G:[ACC: %41.02, LOSS: 90.78]
E: 1, D:[ACC: %80.86, LOSS: 38.33], G:[ACC: %0.39, LOSS: 489.34]
E: 2, D:[ACC: %85.94, LOSS: 31.26], G:[ACC: %0.00, LOSS: 437.58]
E: 3, D:[ACC: %96.24, LOSS: 11.27], G:[ACC: %0.39, LOSS: 487.58]
E: 4, D:[ACC: %94.48, LOSS: 15.40], G:[ACC: %0.00, LOSS: 404.61]
E: 5, D:[ACC: %98.68, LOSS: 6.07], G:[ACC: %0.00, LOSS: 618.42]
E: 6, D:[ACC: %99.71, LOSS: 2.04], G:[ACC: %0.00, LOSS: 694.08]
E: 7, D:[ACC: %99.51, LOSS: 3.03], G:[ACC: %0.00, LOSS: 741.18]
E: 8, D:[ACC: %99.80, LOSS: 1.90], G:[ACC: %0.00, LOSS: 689.42]
E: 9, D:[ACC: %99.71, LOSS: 2.03], G:[ACC: %0.00, LOSS: 726.29]
E: 10, D:[ACC: %93.36, LOSS: 19.07], G:[ACC: %0.00, LOSS: 802.16]
E: 11, D:[ACC: %89.50, LOSS: 24.60], G:[ACC: %0.39, LOSS: 747.72]
E: 12, D:[ACC: %95.80, LOSS: 11.58], G:[ACC: %2.73, LOSS: 520.00]
E: 13, D:[ACC: %88.23, LOSS: 29.45], G:[ACC: %0.39, LOSS: 700.10]
E: 14, D:[ACC: %95.36, LOSS: 12.54], G:[ACC: %1.95, LOSS: 545.11]
E: 15, D:[ACC: %94.24, LO

E: 124, D:[ACC: %79.69, LOSS: 42.07], G:[ACC: %4.30, LOSS: 266.78]
E: 125, D:[ACC: %82.62, LOSS: 40.16], G:[ACC: %7.03, LOSS: 250.36]
E: 126, D:[ACC: %74.76, LOSS: 51.00], G:[ACC: %1.17, LOSS: 274.94]
E: 127, D:[ACC: %80.22, LOSS: 43.27], G:[ACC: %7.42, LOSS: 216.89]
E: 128, D:[ACC: %77.15, LOSS: 46.80], G:[ACC: %2.73, LOSS: 260.08]
E: 129, D:[ACC: %77.78, LOSS: 46.07], G:[ACC: %5.08, LOSS: 258.60]
E: 130, D:[ACC: %75.93, LOSS: 49.93], G:[ACC: %6.64, LOSS: 239.78]
E: 131, D:[ACC: %74.71, LOSS: 52.03], G:[ACC: %2.73, LOSS: 267.85]
E: 132, D:[ACC: %76.81, LOSS: 48.81], G:[ACC: %4.30, LOSS: 243.58]
E: 133, D:[ACC: %82.71, LOSS: 38.88], G:[ACC: %3.91, LOSS: 277.99]
E: 134, D:[ACC: %80.08, LOSS: 43.17], G:[ACC: %3.52, LOSS: 295.00]
E: 135, D:[ACC: %75.73, LOSS: 50.13], G:[ACC: %10.94, LOSS: 205.32]
E: 136, D:[ACC: %78.76, LOSS: 44.70], G:[ACC: %5.08, LOSS: 247.17]
E: 137, D:[ACC: %80.81, LOSS: 42.38], G:[ACC: %5.86, LOSS: 256.16]
E: 138, D:[ACC: %80.22, LOSS: 42.78], G:[ACC: %7.42, LOSS: 27

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
generator.save('./mnist-gen.h5')

In [ ]:
discriminator.save('./mnist-disc.h5')

In [ ]:
gan.save('./mnist-gan.h5')